In [257]:
import yfinance 
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

In [258]:
data = yfinance.download("BTC-USD",start="2014-01-01", period="max", auto_adjust=True, interval="1d")

[*********************100%***********************]  1 of 1 completed


In [259]:
data

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100
...,...,...,...,...,...
2025-09-16,116843.187500,117005.273438,114813.093750,115423.757812,45781744593
2025-09-17,116468.507812,117328.609375,114794.976562,116840.507812,60528025996


In [260]:
FONT_FAMILY = 'ui-sans-serif,-apple-system,system-ui,Segoe UI,Helvetica,Apple Color Emoji,Arial,sans-serif,Segoe UI Emoji,Segoe UI Symbol'

In [261]:
import numpy as np


def plot_chat_Candlestick(data, symbol='BTC-USD', scale=None):

    if scale == 'log':
        data = data.apply(lambda x: np.log1p(x))
        
    elif scale == 'sqrt':
        data = data.apply(lambda x: np.sqrt(x))

    
    fig = go.Figure([
        go.Candlestick(
            x=data.index,
            open=data['Open'].values.flatten(),   # transforma em 1D
            high=data['High'].values.flatten(),
            low=data['Low'].values.flatten(),
            close=data['Close'].values.flatten(),
        )
    ])
    fig.update_traces(
        hoverlabel = dict(
            font = dict(
                size = 16
            )
        )
    )
    fig.update_layout(
        title = dict(
            text = f'Candlestick {symbol}',
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            )
        ),
        xaxis = dict(
            title = dict(
                text = 'Date'
            )
        ),
        font = dict(
            family = FONT_FAMILY
        ),
        margin = dict(
            l = 10,
            r = 10,
            t = 40,
            b = 40
        ),
        template = 'simple_white',
        hovermode = 'x unified',
        xaxis_rangeslider_visible=False

    )
    return fig

In [262]:
plot_chat_Candlestick(data).show()

In [263]:
def volatility(data):
    # daily Volatility : Close - Open
    
    data['volatility'] = data['High'] - data['Low']
 
    fig = go.Figure([
        go.Scatter(
            x=data.index, 
            y=data['volatility'], 
            mode='lines'
        )
    ])

    title_text = 'Daily Volatility'
    fig.update_layout(
        title = dict(
            text = title_text,
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            ) 
        ),
        xaxis = dict(
            title = dict(
                text = 'Date'
            )
        ),
        yaxis = dict(
            title = dict(
                text = 'Volatility'
            )
        ),
        font = dict(
            family = FONT_FAMILY,
        ),
        template = 'simple_white'
    )
    return fig

In [264]:
volatility(data).show()

In [265]:
def return_rate(data, window=0):

    if window == 0:
        data['return_rate'] = data['Close'] - data['Open']
    else:
        data['return_rate'] = data['Close'].pct_change(window)
        data = data.dropna()

    fig = go.Figure([
        go.Scatter(
            x=data.index, 
            y=data['return_rate'], 
            mode='lines'
        )
    ])

    fig.update_traces(
        hoverlabel = dict(
            bgcolor = 'white',
            font = dict(
                size = 16
            )
        ),
        hovertemplate = '%{x} <br> Return Rate : %{y}<extra></extra>'
    )
    
    title_text = 'Return Rate, the last '+ str(window) + ' days'

    fig.add_hline(
        y=0, 
        line_width=2,
        line_dash ='solid', 
        line_color="black"
    )
    fig.update_layout(
        title = dict(
            text = title_text,
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            ) 
        ),
        xaxis = dict(
            title = dict(
                text = 'Date'
            )
        ),
        yaxis = dict(
            title = dict(
                text = 'Return Rate'
            )
        ),
        
        font = dict(
            family = FONT_FAMILY,
        ),
        template = 'simple_white',
        hovermode = 'x'
    )
    return fig

In [266]:
return_rate(data, 40).show()

In [267]:
import statsmodels.api as sm
from plotly.subplots import make_subplots

def decompose_time_series(data, model='additive', freq=None):
    # model = 'additive' or 'multiplicative'
    # freq = None (let statsmodels decide) or integer (number of periods in a season)
    
    result = sm.tsa.seasonal_decompose(data['Close'].values.ravel(), model=model, period=freq)
    
    fig = go.Figure()
    # Transformar em DataFrame
    df_decomp = pd.DataFrame({
        "observed": result.observed,
        "trend": result.trend,
        "seasonal": result.seasonal,
        "resid": result.resid
    }, index=data.index)

    # Criar subplots com Plotly
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                        subplot_titles=("Observed", "Trend", "Seasonal", "Residuals"))

    fig.add_trace(
        go.Scatter(
            x=df_decomp.index, 
            y=df_decomp["observed"], 
            name="Observed"
        ), row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_decomp.index, 
            y=df_decomp["trend"], 
            name="Trend"
        ), row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_decomp.index, 
            y=df_decomp["seasonal"], 
            name="Seasonal"
            ), row=3, col=1
        )
    
    fig.add_trace(
        go.Scatter(
            x=df_decomp.index, 
            y=df_decomp["resid"], 
            name="Residuals"
        ), row=4, col=1
    )

    fig.update_layout(height=900, width=1000, title_text="Time Series Decomposition (Plotly)")
    fig.update_layout(
        title = dict(
            text = 'Decomposition of the time series',
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            )
        ),
        font = dict(
            family = FONT_FAMILY
        ),
        template = 'simple_white',
        dragmode=None
    )
    return fig

In [268]:
decompose_time_series(data, model='multiplicative', freq=5).show()

In [269]:
month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

def pct_change_data_monthly(data):
    series = data[['Close', 'Open']]
    series.columns = series.columns.droplevel(1)
    series = series.apply(lambda x: round(x, 2))
    
    # Resample para o último dia do mês
    closed = series['Close'].resample('ME').last()
    opened = series['Open'].resample('ME').first()
    
    # Calcular a taxa de retorno mensal
    series = ((closed - opened) / opened).reset_index(name='return_rate') 
    series = series.set_index('Date')

    # Porcentagem
    series = series.apply(lambda x: x*100)

    
    series['month_name'] = series.index.month_name()
    series['year']       = series.index.year

    # Ordenar 
    series['month_name'] = pd.Categorical(series['month_name'], categories=month_names, ordered=True)
    series = series.sort_values(by='month_name')
    # Agrupar por mês e ano
    return pd.pivot_table(series, values='return_rate', index='month_name', columns=['year']).apply(lambda x: round(x, 2)), series


In [270]:
subset = pct_change_data_monthly(data)[0] 

C:\Users\mateu\AppData\Local\Temp\ipykernel_7512\3415876939.py:27: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



In [271]:
subset

year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
month_name,,,,,,,,,,,,
January,NaN,-32.13,-14.38,0.70,-27.57,-7.71,29.96,14.21,-16.90,39.83,0.72,9.61
February,NaN,17.24,18.51,21.53,1.57,11.39,-7.99,36.31,12.24,0.04,43.76,-17.61
March,NaN,-3.96,-4.84,-9.17,-32.85,6.53,-25.13,30.47,5.43,23.01,16.62,-2.16
April,NaN,-3.30,7.57,25.77,31.95,30.34,34.51,-2.00,-17.21,2.79,-15.00,14.12
May,NaN,-2.44,18.49,69.58,-18.99,60.24,9.09,-35.31,-15.70,-6.87,11.35,11.07
June,NaN,14.26,26.78,8.41,-14.62,26.16,-3.44,-6.04,-37.77,11.97,-7.13,2.39
July,NaN,8.09,-7.11,15.36,21.35,-6.59,23.81,18.81,17.74,-4.08,3.10,8.04
August,NaN,-19.19,-7.87,63.81,-9.42,-4.43,3.16,13.76,-14.08,-11.29,-8.75,-6.48
September,-16.94,2.52,5.94,-7.72,-5.95,-13.88,-7.66,-7.03,-3.09,3.99,7.39,7.28


In [272]:
import plotly.express as px 
def plot_pct_change_data_monthly(subset):
    subset = subset.fillna('0')
    fig = go.Figure([
        go.Heatmap(
            z = subset.values, 
            x = subset.columns, 
            y = subset.index,
            text = subset.values,
            zmid=0,
            colorscale="RdYlGn",
            showscale=False,
        )
    ])
    fig.update_traces(
        texttemplate='%{text:.2f}%',
    )
    fig.update_layout(
        title = dict(
            text = 'Pct Change Data Monthly',
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            )
        ),
        font = dict(
            family = FONT_FAMILY,
            size = 14
        ),
        template = 'simple_white',
        dragmode=None,
        height=600,
        xaxis = dict(
            tickvals = subset.columns,
            ticktext = subset.columns
        )
    )
    return fig

In [273]:
plot_pct_change_data_monthly(subset).show()

In [274]:
series = pct_change_data_monthly(data)[1]

C:\Users\mateu\AppData\Local\Temp\ipykernel_7512\3415876939.py:27: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



In [275]:
def return_pos_neg_pct_change(series):
    series['is_positive'] = np.where(series['return_rate'] > 0, 1, 0)
    subset = series.groupby('month_name')['is_positive'].value_counts()
    subset = subset.reset_index()
    subset['is_positive'] = subset['is_positive'].apply(lambda x: 'positive' if x == 1 else 'negative')
    return subset

In [276]:
subset = return_pos_neg_pct_change(series)

C:\Users\mateu\AppData\Local\Temp\ipykernel_7512\4167179365.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [277]:
def plot_pos_neg_pct_change(subset):
    fig = go.Figure([])

    for is_pos in subset['is_positive'].unique():
        subset_pos = subset[subset['is_positive'] == is_pos]

        fig.add_trace(
            go.Bar(
                x = subset_pos['month_name'], 
                y = subset_pos['count'], 
                name = is_pos,
                text = subset_pos['count'],
                orientation='v',
                marker=dict(
                    color = '#2a9d8f' if is_pos == 'positive' else '#e63946'
                )
            )
        )

    fig.update_layout(
        title = dict(
            text = 'Positive and Negative Pct Change Data Monthly',
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            )
        ),
        font = dict(
            family = FONT_FAMILY,
            size = 14
        ),
        template = 'simple_white',
        dragmode=None,
    )
    return fig

In [278]:
plot_pos_neg_pct_change(subset).show()

In [279]:
def plot_pie_pos_neg_pct_change(subset):
    fig = go.Figure([
        go.Pie(
            labels = subset['is_positive'], 
            values = subset['count'], 
            textinfo = 'label+value+percent'
        )
    ])
    fig.update_traces(
        marker = dict(
            colors = ['#2a9d8f', '#e63946']
        )
    )

    fig.update_layout(
        title = dict(
            text = 'Positive and Negative Pct Change Data',
            font = dict(
                family = FONT_FAMILY,
                weight = 'bold'
            )
        ),
        font = dict(
            family = FONT_FAMILY,
            size = 14
        ),
        template = 'simple_white',
        dragmode=None,
        height=600,
        width=600
    )
    return fig

In [280]:
plot_pie_pos_neg_pct_change(subset).show()